In [13]:
import numpy as np
import pandas as pd
import nltk

In [14]:
#Download requirements
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pouri\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [15]:
data = pd.read_csv('Data/digikala_comment.csv')
data

,comment
0,نسبت به قیمتش ارزش خرید داره\nجاداره، طراحیش ق...
1,چند ماهی میشه که گرفتمش‌. برای برنامه نویسی و ...
2,پراید ستون جدید
3,اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره ...
4,گوسی هو اوی p10 lite سیپیو و دوربین و رمش از ا...
...,...
235,پوشش دهی صفر.اصلا پیشنهاد نمیکنم
236,نصب این فن خیلی راحته و دردسر زیادی نداره درض...
237,بی کیفیت
238,سلام ٬ چندماهی میشه این پاور بانک رو تهیه کردم...


In [16]:
# Sentence Tokenizing
from nltk.tokenize import sent_tokenize

def sentence_tokenizing(text):
    return sentence_tokenizing(text)

data['comment'] = data.apply(lambda text:sent_tokenize(text['comment']), axis=1)
data

,comment
0,[نسبت به قیمتش ارزش خرید داره\nجاداره، طراحیش ...
1,"[چند ماهی میشه که گرفتمش‌., برای برنامه نویسی ..."
2,[پراید ستون جدید]
3,[اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره...
4,[گوسی هو اوی p10 lite سیپیو و دوربین و رمش از ...
...,...
235,[پوشش دهی صفر.اصلا پیشنهاد نمیکنم]
236,[نصب این فن خیلی راحته و دردسر زیادی نداره در...
237,[بی کیفیت]
238,[سلام ٬ چندماهی میشه این پاور بانک رو تهیه کرد...
